In [ ]:
# library
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import scipy



# load Neural Network Model Library => condition 3 of assignment
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
# path
train_path = "../../dataset/cifar10/train/"
test_path = "../../dataset/cifar10/test/"

batch_size_num = 100
epochs_num = 30

In [ ]:
# Set generator with rescaler(1./255) -> condition 2 of assignment
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.1)

test_datagen = ImageDataGenerator(rescale=1./255)

# make a loading data flow from path. It generates data at each batch sizes -> condition 1 of assignment
train_generator = train_datagen.flow_from_directory(train_path,
                                                    classes=sorted(os.listdir(train_path)),
                                                    batch_size = batch_size_num,
                                                    target_size = (256, 256),
                                                    subset="training",
                                                    class_mode='categorical')

valid_generator = train_datagen.flow_from_directory(train_path, 
                                                    classes=sorted(os.listdir(train_path)), 
                                                    batch_size = batch_size_num, 
                                                    target_size=(256, 256), 
                                                    subset="validation",
                                                    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(test_path, 
                                                  classes=sorted(os.listdir(test_path)),
                                                  batch_size = batch_size_num,
                                                  target_size = (256, 256),
                                                  class_mode='categorical')

print()
# check shape of data shape
print("check shape of data shape")
for x_data, class_data in train_generator:
    print(f"input data shape from train_generator: {x_data.shape}")
    print(f"class data shape from train_generator: {class_data.shape}")
    break

for x_data, class_data in valid_generator:
    print(f"input data shape from valid_generator: {x_data.shape}")
    print(f"class data shape from valid_generator: {class_data.shape}")
    break
    
for x_data, class_data in test_generator:
    print(f"input data shape from test_generator: {x_data.shape}")
    print(f"class data shape from test_generator: {class_data.shape}")
    break    



In [ ]:
# set neural model architecture (transform AlexNet architecture)

model = Sequential()

# 첫 번째 컨볼루션 레이어
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두 번째 컨볼루션 레이어
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 완전 연결 레이어
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))

# 출력 레이어
model.add(Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# 모델 요약
model.summary()

In [ ]:
# train with fit_generator method
# steps_per_epoch => same with iterations(train_data_size//batch_size) in model.fit
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//valid_generator.batch_size,
    shuffle=True,
    epochs=epochs_num)

In [ ]:
# accuracy plot

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, label="Training acc")
plt.plot(epochs, val_acc, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.ylim(0,1)
# plt.savefig(f'./result_plot/batch_{batch_size_num}_epoch_{epochs_num}_acc_plot.png')
plt.show()

In [ ]:
# loss plot
acc = history.history['loss']
val_acc = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, label="Training loss")
plt.plot(epochs, val_acc, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.ylim(0, 3)
# plt.savefig(f'./result_plot/batch_{batch_size_num}_epoch_{epochs_num}_loss_plot.png')
plt.show()

In [ ]:
# model evaluation
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=test_generator.n//test_generator.batch_size)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
model.save('../../pretrained_model_repo/model_LeNet_cifar10.h5')